In [19]:
import os
import shutil
import glob
import numpy as np 
import pandas as pd
import pretty_midi
import pypianoroll
import tables
from music21 import converter, instrument, note, chord, stream
import music21
import librosa
import librosa.display
import matplotlib.pyplot as plt
from keras.utils import np_utils
import json
import IPython.display
from datetime import datetime
import random

import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from tqdm.notebook import tqdm, trange  

import random
import itertools
root_dir = r'C:\Users\ahdz3\Desktop\jazzmidis'
data_dir = root_dir + '\Jazz Midi'
# music_dataset_lpd_dir = root_dir + '/Music Dataset/midis/lmd_matched'

ImportError: DLL load failed while importing utilsextension: The specified module could not be found.

In [3]:
print(root_dir)
midi_info = pd.read_csv(root_dir + '\Jazz-midi.csv')
# midi_info.head()

C:\Users\ahdz3\Desktop\jazzmidis


In [ ]:
# view which tracks are in our jazz midi files
j = 0

for msd_file_name in midi_info['Name']:


  # create path to midi file
  midi_path = os.path.join(data_dir, msd_file_name)
  
  # continue if file is corrupted
  try:
    multitrack = pypianoroll.read(midi_path)
  except:
    print(msd_file_name, ' is corrupted')
    continue

  multitrack.set_resolution(2).pad_to_same()

  print(j, ': ', multitrack.tracks, '\n\n')
  j += 1


In [ ]:
combined_pianorolls = []
i = 0

# change root_dir to training set afterwards
for msd_file_name in midi_info['Name']:

   # create path to midi file
  midi_path = os.path.join(data_dir, msd_file_name)
  
  # continue if file is corrupted
  try:
    multitrack = pypianoroll.read(midi_path)
  except:
    print(msd_file_name, ' is corrupted')
    continue

  multitrack.set_resolution(2).pad_to_same()

  # Piano, Guitar, Bass, Strings, Drums
  # Splitting into different parts

  # RESOLVE: original code uses LPD-5 standardized pianorolls that have all 5 parts

  parts = {'piano_part': None, 'guitar_part': None, 'bass_part': None, 'strings_part': None, 'drums_part': None}
  song_length = None
  empty_array = None
  has_empty_parts = False
  for track in multitrack.tracks:
    if 'Drums' in track.name:
      parts['drums_part'] = np.array(track.pianoroll)
    if 'Piano' in track.name:
      parts['piano_part'] = np.array(track.pianoroll)
    if 'Guitar' in track.name:
      parts['guitar_part'] = np.array(track.pianoroll)
    if 'Bass' in track.name:
      parts['bass_part'] = np.array(track.pianoroll)
    if 'Strings' in track.name:
      parts['strings_part'] = np.array(track.pianoroll)
    if track.pianoroll.shape[0] > 0:
      empty_array = np.zeros_like(track.pianoroll)


  for k,v in parts.items():
    if (v is None) or (v.shape[0] == 0):
      parts[k] = empty_array.copy()
      has_empty_parts = True

  
  # These contain velocity information - the force with which the notes are hit - which can be standardized to 0/1 if we want (to compress)
  if has_empty_parts == False:
    combined_pianorolls.append(combined_pianoroll)
    i+=1
    print(i)

  # Stack all together - Piano, Guitar, Bass, Strings, Drums
  combined_pianoroll = torch.tensor([parts['piano_part'], parts['guitar_part'], parts['bass_part'], parts['strings_part'], parts['drums_part']])

  if i == 1000:
    break

In [4]:
combined_pianorolls = []
i = 0

# change root_dir to training set afterwards
for msd_file_name in midi_info['Name']:

   # create path to midi file
  midi_path = os.path.join(data_dir, msd_file_name)
  
  # continue if file is corrupted
  try:
    multitrack = pypianoroll.read(midi_path)
  except:
    print(msd_file_name, ' is corrupted')
    continue

  multitrack.set_resolution(2).pad_to_same()

  # Piano, Guitar, Bass, Strings, Drums
  # Splitting into different parts

  # RESOLVE: original code uses LPD-5 standardized pianorolls that have all 5 parts

  parts = {'piano_part': None, 'guitar_part': None, 'bass_part': None, 'strings_part': None, 'drums_part': None}
  song_length = None
  empty_array = None
  has_empty_parts = False
  for track in multitrack.tracks:
    if 'Drums' in track.name:
      parts['drums_part'] = track.pianoroll
    if 'Piano' in track.name:
      parts['piano_part'] = track.pianoroll
    if 'Guitar' in track.name:
      parts['guitar_part'] = track.pianoroll
    if 'Bass' in track.name:
      parts['bass_part'] = track.pianoroll
    if 'Strings' in track.name:
      parts['strings_part'] = track.pianoroll
    if track.pianoroll.shape[0] > 0:
      empty_array = np.zeros_like(track.pianoroll)


  for k,v in parts.items():
    if (v is None) or (v.shape[0] == 0):
      parts[k] = empty_array.copy()
      has_empty_parts = True

  # Stack all together - Piano, Guitar, Bass, Strings, Drums
  combined_pianoroll = torch.tensor([parts['piano_part'], parts['guitar_part'], parts['bass_part'], parts['strings_part'], parts['drums_part']])

  # These contain velocity information - the force with which the notes are hit - which can be standardized to 0/1 if we want (to compress)
  if has_empty_parts == False:
    combined_pianorolls.append(combined_pianoroll)
    i+=1
    print(i)

  if i == 1000:
    break

C:\Users\ahdz3\AppData\Local\Temp/ipykernel_5148/552587433.py:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  combined_pianoroll = torch.tensor([parts['piano_part'], parts['guitar_part'], parts['bass_part'], parts['strings_part'], parts['drums_part']])


1
2


c:\Users\ahdz3\mambaforge\lib\site-packages\pretty_midi\pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


ThePrincess.mid  is corrupted
3
4
5
DarnThatDream.mid  is corrupted
6
7
8
9
10
Moment.mid  is corrupted
11
12
13
14
15
StoriesOfThePaintedDesert.mid  is corrupted
16
StTropez.mid  is corrupted
17
18
19
NativeSonsOfADistantLand.mid  is corrupted
Lakes.mid  is corrupted
RoadWarriors.mid  is corrupted
CantilopeIsland.mid  is corrupted
20
21
22
23
24
25
26
27
28
29
30
31
AnyWayYouWantIt.mid  is corrupted
AffairInSanMiguel.mid  is corrupted
32
JamaicanNights.mid  is corrupted
Destiny.mid  is corrupted
33
34
BigSky.mid  is corrupted
35
36
37
38
CurvesAhead.mid  is corrupted
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
BraveNewWorld.mid  is corrupted
65
66


In [7]:
pianoroll_lengths = [e.size()[1] for e in combined_pianorolls]
combined_pianorolls = torch.hstack(combined_pianorolls)

In [8]:
torch.save(combined_pianorolls, os.path.join(root_dir, 'jazz_rolls_vae.pt'))
pianoroll_lengths = torch.tensor(pianoroll_lengths)
torch.save(pianoroll_lengths, os.path.join(root_dir, 'jazz_rolls_vae_lengths.pt'))